In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

import numpy as np
from pandas_summary import DataFrameSummary
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [2]:
# %pip install pandas_summary

In [3]:
# # split train y val
# n_split = int(df_full.shape[0]*0.2)
# df_train = df_full[n_split:]
# df_val = df_full[:n_split]
# len(df_train), len(df_val)

In [4]:
df_full = pd.read_feather('../data/full_data.fth')
df_full = df_full.sort_values(by='Date_rating')
df_full.head(2)

,user_id,Full Name,year of birth,Gender,Occupation,Active Since,Trabajador,Antiguedad,Horoscopo Activacion,movie_id,rating,Date_rating,Name_movie,Release Date,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,Antiguedad_pelicula
25420,259,Allen Oliver,1977,M,student,1997-09-20 03:05:10,0,0,9,255,4,1997-09-20 03:05:10,My Best Friend's Wedding (1997),1997-06-20,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,27564.0
25461,259,Allen Oliver,1977,M,student,1997-09-20 03:05:10,0,0,9,286,4,1997-09-20 03:05:27,"English Patient, The (1996)",1996-11-15,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,27347.0


In [5]:
df_full.columns

Index(['user_id', 'Full Name', 'year of birth', 'Gender', 'Occupation',
       'Active Since', 'Trabajador', 'Antiguedad', 'Horoscopo Activacion',
       'movie_id', 'rating', 'Date_rating', 'Name_movie', 'Release Date',
       'unknown', 'Action', 'Adventure', 'Animation', 'Children's', 'Comedy',
       'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
       'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western',
       'Antiguedad_pelicula'],
      dtype='object')

## Separo en categoricas y numericas, los genres los dejare como binarios

In [6]:
cat_vars = ['Gender', 'Occupation', 'Horoscopo Activacion']
df_full[cat_vars].head()

,Gender,Occupation,Horoscopo Activacion
25420,M,student,9
25461,M,student,9
25449,M,student,9
25439,M,student,9
25459,M,student,9


In [7]:
contin_vars = [ 'year of birth','Trabajador','Antiguedad', 'Antiguedad_pelicula']

df_full[contin_vars].head()

,year of birth,Trabajador,Antiguedad,Antiguedad_pelicula
25420,1977,0,0,27564.0
25461,1977,0,0,27347.0
25449,1977,0,0,27571.0
25439,1977,0,0,13879.0
25459,1977,0,0,23741.0


In [8]:
DataFrameSummary(df_full[contin_vars]).summary().loc[['uniques', 'types', 'missing']].T

,uniques,types,missing
year of birth,61,numeric,0
Trabajador,2,bool,0
Antiguedad,196,numeric,0
Antiguedad_pelicula,240,numeric,9


# Normalizacion y labelencode

In [9]:
from sklearn_pandas import DataFrameMapper

In [10]:
cat_maps = [(o, LabelEncoder()) for o in cat_vars]
contin_maps = [([o], StandardScaler()) for o in contin_vars]

In [11]:
mapper_cat = DataFrameMapper(cat_maps)
_ = mapper_cat.fit(df_full)

In [12]:
N = 100
print(df_full['Occupation'].values[30:35])
print(mapper_cat.transform(df_full)[:, 2][30:35])

['student' 'student' 'student' 'student' 'student']
[4 4 4 4 4]


In [13]:
N = 100
print(df_full['user_id'].values[30:35])
print(mapper_cat.transform(df_full)[:, 2][30:35])

[712 712 712 712 712]
[4 4 4 4 4]


In [14]:
mapper_cont = DataFrameMapper(contin_maps)
_ = mapper_cont.fit(df_full)

In [15]:
N = 5
print(df_full['Antiguedad_pelicula'].values[:N])
print(mapper_cont.transform(df_full)[:, 0][:N])

[27564. 27347. 27571. 13879. 23741.]
[1.03522441 1.03522441 1.03522441 1.03522441 1.03522441]


In [16]:
# La hacemos con todas ahora
df_full[cat_vars] = mapper_cat.transform(df_full)
df_full[contin_vars] = mapper_cont.transform(df_full)

In [17]:
df_full = df_full.reset_index()
df_full.head()

,index,user_id,Full Name,year of birth,Gender,Occupation,Active Since,Trabajador,Antiguedad,Horoscopo Activacion,movie_id,rating,Date_rating,Name_movie,Release Date,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,Antiguedad_pelicula
0,25420,259,Allen Oliver,1.035224,1,19,1997-09-20 03:05:10,-0.112964,-1.453686,4,255,4,1997-09-20 03:05:10,My Best Friend's Wedding (1997),1997-06-20,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0.657134
1,25461,259,Allen Oliver,1.035224,1,19,1997-09-20 03:05:10,-0.112964,-1.453686,4,286,4,1997-09-20 03:05:27,"English Patient, The (1996)",1996-11-15,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0.615426
2,25449,259,Allen Oliver,1.035224,1,19,1997-09-20 03:05:10,-0.112964,-1.453686,4,298,4,1997-09-20 03:05:54,Face/Off (1997),1997-06-27,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0.658480
3,25439,259,Allen Oliver,1.035224,1,19,1997-09-20 03:05:10,-0.112964,-1.453686,4,185,4,1997-09-20 03:06:21,Psycho (1960),1960-01-01,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,-1.973156
4,25459,259,Allen Oliver,1.035224,1,19,1997-09-20 03:05:10,-0.112964,-1.453686,4,173,4,1997-09-20 03:07:23,"Princess Bride, The (1987)",1987-01-01,0,1,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,-0.077656


In [18]:
df_full.to_feather('../data/full_normalized_data.fth')

In [19]:
df_full[df_full.movie_id == 267]

,index,user_id,Full Name,year of birth,Gender,Occupation,Active Since,Trabajador,Antiguedad,Horoscopo Activacion,movie_id,rating,Date_rating,Name_movie,Release Date,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,Antiguedad_pelicula
5542,30917,297,Dolores Klien,0.343336,0,4,1997-09-22 18:44:52,-0.112964,-1.421867,4,267,3,1997-09-28 01:12:19,unknown,NaT,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
5619,59513,532,Frank Roberts,1.121710,1,19,1997-09-20 20:29:47,-0.112964,-1.453686,4,267,3,1997-09-28 10:09:08,unknown,NaT,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
6107,434,5,Janice Mccullough,-0.002608,0,14,1997-09-30 15:57:44,-0.112964,-1.294592,4,267,4,1997-09-30 15:57:44,unknown,NaT,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
6509,88295,833,Sammy Hoover,-0.089094,1,21,1997-09-23 17:24:47,-0.112964,-1.405958,4,267,1,1997-09-30 21:41:09,unknown,NaT,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
6513,47507,422,Isaac Landry,0.602794,1,6,1997-09-24 19:32:03,-0.112964,-1.390048,4,267,4,1997-09-30 21:46:26,unknown,NaT,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
6864,112,1,Robert Stanley,0.775766,1,20,1997-09-22 21:57:58,-0.112964,-1.421867,4,267,4,1997-10-01 08:02:35,unknown,NaT,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
6869,34464,319,Todd Baughman,-0.435038,1,15,1997-10-01 12:08:10,-0.112964,-1.278682,5,267,4,1997-10-01 12:08:10,unknown,NaT,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
7387,26236,268,Edward Tomlinson,0.775766,1,5,1997-09-26 20:41:17,-0.112964,-1.358229,4,267,3,1997-10-01 21:41:17,unknown,NaT,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
7752,13416,130,Luis Lineberger,1.121710,1,13,1997-09-22 18:33:59,-0.112964,-1.421867,4,267,5,1997-10-02 14:07:19,unknown,NaT,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
